Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [73]:
from random import random, seed
from itertools import product
import numpy as np

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [74]:
UNIVERSE_SIZE = 1000
NUM_SETS = 100
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [75]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.power(SETS.sum(axis=1), 1.1)

## Helper Functions

In [76]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [77]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
valid(solution), cost(solution)

(np.True_, np.float64(33635.922665122314))

In [78]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < .5
valid(solution), cost(solution)

(np.True_, np.float64(15859.594977250465))

My Solutions

The first following solution is structured in two parts:
1) Find the set that has the largest amout of elements
2) Iterate over the other sets and include in the min_set list all the sets that cover a missing element of the previous min_set sets.

In [79]:
min_sets=[]
not_covered = np.ones(UNIVERSE_SIZE, dtype=bool)

# Search for the set with the highest number of elements
best_set = None
best_set_covered = set()

for i in range(len(SETS)):
    covered_by_set = np.where(SETS[i] & not_covered)[0]
    if len(covered_by_set) > len(best_set_covered):
        best_set = i
        best_set_covered = covered_by_set

min_sets.append(best_set)
not_covered[best_set_covered] = False

# Add new sets that have some value of the not covered
while np.any(not_covered):
    for elem in np.where(not_covered)[0]:
        found = False
        for i in range(len(SETS)):
            if i in min_sets:
                continue
            if SETS[i][elem]:
                min_sets.append(i)
                not_covered[np.where(SETS[i])[0]] = False
                found = True
                break
        if found:
            break
        
print("Number of elements in min_sets:", len(min_sets))
print("Indices of the set taken and order:", min_sets)
print("Is the solution valid?", valid(min_sets))
print("Cost of the solution:", cost(min_sets))


Number of elements in min_sets: 25
Indices of the set taken and order: [26, 1, 2, 5, 12, 14, 6, 7, 3, 10, 8, 16, 15, 13, 19, 11, 0, 4, 9, 23, 27, 25, 17, 18, 34]
Is the solution valid? True
Cost of the solution: 8430.521054319439


The second following solution takes only the sets with the greatest amout of elements until all elements are taken.

In [80]:
# Greedy initial solution
min_sets = []
not_covered = np.ones(UNIVERSE_SIZE, dtype=bool)

while np.any(not_covered):
    best_set = None
    max_covered = 0
    for i in range(len(SETS)):
        if i in min_sets:
            continue
        covered_by_set = np.where(SETS[i] & not_covered)[0]
        if len(covered_by_set) > max_covered:
            best_set = i
            max_covered = len(covered_by_set)
    if best_set is not None:
        min_sets.append(best_set)
        not_covered[np.where(SETS[best_set])[0]] = False

print("Number of elements in min_sets:", len(min_sets))
print("Indices of the set taken and order:", min_sets)
print("Is the solution valid?", valid(min_sets))
print("Cost of the solution:", cost(min_sets))


Number of elements in min_sets: 18
Indices of the set taken and order: [26, 68, 17, 69, 4, 3, 54, 11, 58, 52, 7, 31, 82, 99, 33, 89, 42, 2]
Is the solution valid? True
Cost of the solution: 6231.286338305842


This last solution is a ibrid version of teh previous two. In this case I take firstly the set with the largest number of elements and then I take the largest sets that cover all not_covered elements.

In [81]:
min_sets = []
not_covered = np.ones(UNIVERSE_SIZE, dtype=bool)

# Step 1: Select the set with the largest number of elements
best_set = None
best_set_covered = set()

for i in range(len(SETS)):
    covered_by_set = np.where(SETS[i] & not_covered)[0]
    if len(covered_by_set) > len(best_set_covered):
        best_set = i
        best_set_covered = covered_by_set

min_sets.append(best_set)
not_covered[best_set_covered] = False

# Step 2: For each uncovered element, find the largest set that contains that element
while np.any(not_covered):
    for elem in np.where(not_covered)[0]:
        best_set = None
        best_set_covered = set()
        for i in range(len(SETS)):
            if i in min_sets:
                continue
            if SETS[i][elem]:
                covered_by_set = np.where(SETS[i] & not_covered)[0]
                if len(covered_by_set) > len(best_set_covered):
                    best_set = i
                    best_set_covered = covered_by_set
        if best_set is not None:
            min_sets.append(best_set)
            not_covered[np.where(SETS[best_set])[0]] = False

print("Number of elements in min_sets:", len(min_sets))
print("Indices of the set taken and order:", min_sets)
print("Is the solution valid?", valid(min_sets))
print("Cost of the solution:", cost(min_sets))

Number of elements in min_sets: 18
Indices of the set taken and order: [26, 91, 95, 86, 24, 51, 17, 40, 69, 63, 28, 74, 2, 83, 21, 16, 72, 62]
Is the solution valid? True
Cost of the solution: 6170.066480370011


After some tries, we can see that some the last and the second solutions have better result in comparison with the first one. 